In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [8]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

def preprocess_embedding(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
def plotHistory(history):
        plt.plot(history.history['accuracy'])
        plt.title('accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

In [24]:
data = [["A", "B", "C", "A", "B", "C", "A", "B", "C", "A", "B", "C", "A", "B", "C"]
       ,["D", "B", "C", "A", "B", "D", "A", "D", "C", "A", "B", "C", "A", "B", "D"]
       ,["C", "B", "C", "A", "B", "D", "A", "C", "C", "A", "B", "C", "A", "B", "C"]]
       
target = ["A", "B", "C"]

print(target)
print(data)

pad_data, pad_target, data_tokenizer, target_tokenizer =  preprocess_embedding(data, target)

max_pad_data_sequence_length = pad_data.shape[1]
max_pad_target_sequence_length = pad_target.shape[1]
data_size = len(data_tokenizer.word_index)+1
target_size = len(target_tokenizer.word_index)+1

#print(pad_data_length,pad_target)
#print(data_size,target_size)


print('Data Preprocessed')
print("Max data length:", max_pad_data_sequence_length)
print("Max target length:", max_pad_target_sequence_length)
print("Data vocabulary size:", data_size)
print("Target vocabulary size:", target_size)



['A', 'B', 'C']
[['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'], ['D', 'B', 'C', 'A', 'B', 'D', 'A', 'D', 'C', 'A', 'B', 'C', 'A', 'B', 'D'], ['C', 'B', 'C', 'A', 'B', 'D', 'A', 'C', 'C', 'A', 'B', 'C', 'A', 'B', 'C']]
Data Preprocessed
Max data length: 15
Max target length: 1
Data vocabulary size: 5
Target vocabulary size: 4


In [25]:
print("pad_data shape",pad_data.shape[1])
print("data size",data_size)


pad_data shape 15
data size 5


In [26]:
# see https://stackoverflow.com/questions/62253289/valueerror-data-cardinality-is-ambiguous
padded=pad_sequences(pad_data)
X = np.expand_dims(pad_data, axis = 0)
#print(X)
print('Shape of X is ', X.shape)
y = np.array(pad_target)
y = y.reshape(1,-1)

print('Shape of y is', y.shape)

model = Sequential()
#model.add(Embedding(input_dim=data_size,output_dim=128,input_length=X.shape[2] ))
model.add(LSTM(data_size, return_sequences=True, input_shape=(None, X.shape[2])))
#model.add(Dense(data_size, activation='relu', input_shape=(X.shape[1], X.shape[2])))
#model.add(Dense(data_size, activation='relu'))
model.add(LSTM(50, activation='relu')),
#model.add(Dense(target_size,activation='softmax'))
#model.add(Dense(1, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

model.compile (
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)
   

history=model.fit(x=X, y=y, epochs=100,verbose=False)
#plotHistory(history)
print(X)
predchance = model.predict(X)
print(predchance)
max_n = np.argmax(predchance)
print("predicted target ", target_tokenizer.index_word[max_n])
#pred = layer.get_vocabulary()[max_n+1]

Shape of X is  (1, 3, 15)
Shape of y is (1, 3)
[[[2 3 1 2 3 1 2 3 1 2 3 1 2 3 1]
  [4 3 1 2 3 4 2 4 1 2 3 1 2 3 4]
  [1 3 1 2 3 4 2 1 1 2 3 1 2 3 1]]]
1/1 [==============================] - 0s 473ms/step
[[1.4084450e-04 6.5457565e-03 9.9331337e-01]]
predicted target  b


In [27]:
print("done")

done
